<a href="https://colab.research.google.com/github/Hani1-2/40-Algorithms-Every-Programmer-Should-Know/blob/master/ANN_arabic_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
data_path = '/content/drive/MyDrive/FYP-Code/data_prep_ann.json'
with open(data_path, "r") as fp:
      data = json.load(fp)


In [2]:
import numpy as np


In [3]:
### Split the dataset into independent and dependent dataset
X=np.array(data['MFCCs'])
y=np.array(data['labels'])

In [4]:
X.shape

(21952, 40)

In [5]:
y.shape

(21952,)

In [6]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [7]:
y.shape

(21952, 28)

In [8]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
y_train.shape

(17561, 28)

In [10]:
X_train.shape

(17561, 40)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [12]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from keras import layers
from keras import layers
import keras

In [19]:
import keras.backend as K
# define a custom loss function that incorporates the confidence score
def confidence_loss(y_true, y_pred, conf_weight=1.0):
    # compute the cross-entropy loss
    cross_entropy = keras.losses.categorical_crossentropy(y_true, y_pred)

    # compute the confidence score
    confidence = K.max(y_pred, axis=-1)

    # combine the cross-entropy loss and confidence score into a single loss
    loss = cross_entropy + conf_weight * (1 - confidence)

    return loss


In [35]:
# generate predictions and confidence scores
def predict_with_confidence(model, x):
    # generate predictions and confidence scores for each input
    preds = model.predict(x)
    confs = np.max(preds, axis=-1)
    print('preds',preds)
    print('confidence score',confs)
    return preds, confs

In [21]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(28, activation='softmax'))
model.compile(optimizer='adam',
              loss=confidence_loss,
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               10496     
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 28)                1820      
                                                                 
Total params: 53,468
Trainable params: 53,468
Non-trainable params: 0
_________________________________________________________________


In [23]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 50
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/50
543/549 [============================>.] - ETA: 0s - loss: 3.2925 - accuracy: 0.2584
Epoch 1: val_loss improved from inf to 2.02934, saving model to audio_classification.hdf5
549/549 [==============================] - 3s 4ms/step - loss: 3.2788 - accuracy: 0.2605 - val_loss: 2.0293 - val_accuracy: 0.4634
Epoch 2/50
535/549 [============================>.] - ETA: 0s - loss: 1.6958 - accuracy: 0.5453
Epoch 2: val_loss improved from 2.02934 to 1.37214, saving model to audio_classification.hdf5
549/549 [==============================] - 2s 3ms/step - loss: 1.6851 - accuracy: 0.5482 - val_loss: 1.3721 - val_accuracy: 0.6185
Epoch 3/50
539/549 [============================>.] - ETA: 0s - loss: 1.1580 - accuracy: 0.6859
Epoch 3: val_loss improved from 1.37214 to 1.04385, saving model to audio_classification.hdf5
549/549 [==============================] - 2s 3ms/step - loss: 1.1561 - accuracy: 0.6867 - val_loss: 1.0439 - val_accuracy: 0.7094
Epoch 4/50
546/549 [=====================

In [24]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9808699488639832


In [40]:
filename="007.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict(mfccs_scaled_features)
print(predicted_label)
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# prediction_class
y_classes = np.argmax(predicted_label, axis=-1)
print(y_classes,'y_classes')

[-2.6688876e+02  1.6920772e+02  1.5943729e+01  2.9220087e+01
  9.9509230e+00  9.4693556e+00  4.4155707e+00  2.2081059e+01
 -8.4803972e+00 -1.0236913e+01 -9.6969500e+00 -1.3531391e+01
 -1.6008681e-01 -9.6199722e+00  4.0289865e+00 -1.1860269e+01
 -8.8936329e+00 -7.5885506e+00 -1.0422292e+01 -2.9442387e-04
 -7.9004889e+00  1.6884052e+00 -4.2548723e+00 -1.7290313e+00
 -2.5600350e+00 -7.0732765e+00 -3.4278486e+00 -7.8612771e+00
 -2.2507224e+00 -3.7321632e+00 -1.2287372e+00 -2.7821176e+00
 -5.1131840e+00 -2.4848039e+00 -3.3958735e+00 -2.2556162e+00
 -9.0258640e-01  5.2021070e+00  6.4245253e+00  5.1930232e+00]
[[-2.6688876e+02  1.6920772e+02  1.5943729e+01  2.9220087e+01
   9.9509230e+00  9.4693556e+00  4.4155707e+00  2.2081059e+01
  -8.4803972e+00 -1.0236913e+01 -9.6969500e+00 -1.3531391e+01
  -1.6008681e-01 -9.6199722e+00  4.0289865e+00 -1.1860269e+01
  -8.8936329e+00 -7.5885506e+00 -1.0422292e+01 -2.9442387e-04
  -7.9004889e+00  1.6884052e+00 -4.2548723e+00 -1.7290313e+00
  -2.5600350e+00 

In [41]:
data['mapping'][6]

'Rabbil'

### when model predict correct output, it gives 1. confidence. Other than that confidence for the words on which the has not trained is less than 90

In [44]:
filename="004.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=predict_with_confidence(model,mfccs_scaled_features)
# print(predicted_label,'predicted label')
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# prediction_class
y_classes = np.argmax(predicted_label[0], axis=-1)
print(y_classes,'predicted label')

[-3.13894287e+02  1.21448944e+02 -1.58409843e+01  4.44018517e+01
 -4.30620308e+01  1.04552956e+01 -7.70796394e+00  1.93449631e+01
 -1.54493647e+01 -5.64498568e+00 -2.62555623e+00 -5.63413095e+00
 -4.70574617e+00 -1.69100113e+01  2.85181732e+01 -2.02912216e+01
  5.73039150e+00  2.51097989e+00 -1.72864075e+01  3.65360522e+00
 -1.08147669e+01 -1.24511704e-01 -7.63013458e+00 -3.31317997e+00
 -8.56130791e+00  6.79593515e+00  1.12333384e+01  2.01670227e+01
  3.11204929e+01  1.60158119e+01  1.81351357e+01  5.09372234e+00
  2.43120146e+00 -8.13832760e+00 -6.95227206e-01  2.06990838e+00
 -5.34135342e+00  2.19039202e+00 -4.89979362e+00 -1.13116610e+00]
[[-3.13894287e+02  1.21448944e+02 -1.58409843e+01  4.44018517e+01
  -4.30620308e+01  1.04552956e+01 -7.70796394e+00  1.93449631e+01
  -1.54493647e+01 -5.64498568e+00 -2.62555623e+00 -5.63413095e+00
  -4.70574617e+00 -1.69100113e+01  2.85181732e+01 -2.02912216e+01
   5.73039150e+00  2.51097989e+00 -1.72864075e+01  3.65360522e+00
  -1.08147669e+01 -

In [45]:
data['mapping'][3]

'Al-raheemi'

#### for incorrect word

In [43]:
filename="test3.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

# print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
# print(mfccs_scaled_features)
# print(mfccs_scaled_features.shape)
predicted_label=predict_with_confidence(model,mfccs_scaled_features)
# print(predicted_label,'predicted label')
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# prediction_class
y_classes = np.argmax(predicted_label[0], axis=-1)
print(y_classes,'predicted label')

1/1 [==============================] - 0s 45ms/step
preds [[0.00000000e+00 0.00000000e+00 1.78855838e-17 0.00000000e+00
  1.36379804e-38 0.00000000e+00 1.40822305e-30 0.00000000e+00
  8.95489872e-01 7.66893690e-33 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.63906467e-16 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.04510166e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.45481954e-16 0.00000000e+00 2.98184979e-12]]
confidence score [0.8954899]
[8] predicted label
